# Scrape PDFs from the Portuguese Parliament
---
Initial experiments to scrape proposals and voting tables from official PDFs from the Portuguese Parliament.

## Setup

### Import libraries

In [ ]:
import os
os.environ["PATH"] += ":/opt/homebrew/lib:/opt/homebrew/bin/gs"
os.environ["DYLD_LIBRARY_PATH"] = "/opt/homebrew/lib"   # this is needed for MacOS to find the Ghostscript library

In [ ]:
from typing import Tuple
import math
import numpy as np
import camelot
from camelot.handlers import PDFHandler
from langchain.document_loaders import PyPDFLoader
from tqdm.auto import tqdm

### Parameters

In [ ]:
deputies_per_party = {
    "PS": 120,
    "PSD": 77,
    "CH": 12,
    "IL": 8,
    "PCP": 6,
    "BE": 5,
    "PAN": 1,
    "L": 1,
}

In [ ]:
pdf_path = "/Users/adminuser/Downloads/XV_1_151_2023-07-07_ResultadoVotacoes_2023-07-07.pdf"

## Scrape proposals' text

In [ ]:
def is_relevant_proposal(text: str) -> bool:
    """Check if a proposal is relevant for our analysis.

    Args:
        text (str): The text of the proposal.
    
    Returns:
        bool: True if the proposal is relevant, False otherwise.
    """
    if (
        (
            all(party in text for party in deputies_per_party.keys()) 
            or "GOV" in text
        )
        and (
            any(word in text.lower() for word in ["favor", "contra", "abstenção"])
            or any(word in text for word in ["aprovad", "rejeitad"])
        )
    ):
        return True
    return False

In [ ]:
def clean_proposal_text(text: str) -> str:
    """
    Process the dedescription of a proposal, removing unnecessary text
    and symbols.

    Args:
        text (str): The text of the proposal.
    
    Returns:
        str: The processed text of the proposal.
    """
    return text.split("Aprovad")[0].split("Rejeitad")[0].replace("\n", "").replace("  ", " ").replace(";", ".")

In [ ]:
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()
pages

In [ ]:
pages[0].page_content

In [ ]:
pages[2].page_content.split("\uf0de")[1:]

In [ ]:
[text for text in pages[2].page_content.split("\uf0de")[1:] if is_relevant_proposal(text)]

In [ ]:
[clean_proposal_text(text) for text in pages[2].page_content.split("\uf0de")[1:] if is_relevant_proposal(text)]

In [ ]:
proposals = list()
unanimous_idx = list()
for page in pages:
    page_proposals = [text for text in page.page_content.split("\uf0de")[1:]] #if is_relevant_proposal(text)]
    for idx, prop in enumerate(page_proposals):
        if "unanimidade" in prop.lower():
            unanimous_idx.append(len(proposals) + idx)
    proposals.extend(page_proposals) #[clean_proposal_text(text) for text in page_proposals])

In [ ]:
proposals

In [ ]:
unanimous_idx

In [ ]:
len(proposals)

In [ ]:
len(proposals) - len(unanimous_idx)

## Scrape voting tables

In [ ]:
def get_top_mid_coords(bbox: Tuple[float, float, float, float], offset: int = 10) -> Tuple[float, float]:
    """
    Get the coordinates of the middle of the top of a bounding box.

    Args:
        bbox (Tuple[float, float, float, float]): The bounding box.
        offset (int, optional): The offset to add to the y coordinate. Defaults to 10.

    Returns:
        Tuple[float, float]: The coordinates of the middle of the top of the bounding box.
    """
    return ((bbox[0] + bbox[2]) / 2, bbox[3] + offset)

def get_bottom_mid(bbox):
    """
    Get the coordinates of the middle of the bottom of a bounding box.

    Args:
        bbox (Tuple[float, float, float, float]): The bounding box.
    
    Returns:
        Tuple[float, float]: The coordinates of the middle of the bottom of the bounding box.
    """
    return ((bbox[0] + bbox[2]) / 2, bbox[1])

def calc_distance_between_coords(p1: Tuple[float, float], p2: Tuple[float, float]) -> float:
    """
    Calculate the distance between two points.

    Args:
        p1 (Tuple[float, float]): The first point.
        p2 (Tuple[float, float]): The second point.

    Returns:
        float: The distance between the two points.
    """
    return math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)

def get_closest_text(table, htext_objs, initial_offset: int = 0, max_tries: int = 100):
    best_guess = None
    best_guess_length = 0
    offset = initial_offset
    num_tries = 0
    while (best_guess_length < 10) and (num_tries < max_tries):
        min_distance = np.inf
        table_mid = get_top_mid_coords(table._bbox, offset=offset)  # middle of the TOP of the table
        for obj in htext_objs:
            text_mid = get_bottom_mid(obj.bbox)  # middle of the BOTTOM of the text
            d = calc_distance_between_coords(text_mid, table_mid)
            if d < min_distance:
                best_guess = obj.get_text().strip()
                min_distance = d
                best_guess_length = len(best_guess)
        offset += 1
        num_tries += 1
    return best_guess

def get_tables_and_titles(pdf_filename):
    """Here's my hacky code for grabbing tables and guessing at their titles"""
    my_handler = PDFHandler(pdf_filename)  # from camelot.handlers import PDFHandler
    tables = camelot.read_pdf(pdf_filename, pages="all")
    titles = []
    with camelot.utils.TemporaryDirectory() as tempdir:
        for table in tqdm(tables, desc=f"Extracting {tables.n:d} tables"):
            my_handler._save_page(pdf_filename, table.page, tempdir)
            tmp_file_path = os.path.join(tempdir, f'page-{table.page}.pdf')
            layout, dim = camelot.utils.get_page_layout(tmp_file_path)
            htext_objs = camelot.utils.get_text_objects(layout, ltype="horizontal_text")
            titles.append(get_closest_text(table, htext_objs))  # Might be None

    return titles, tables

titles, tables = get_tables_and_titles(pdf_path)
for title, table in zip(titles, tables):
    print(title)
    display(table.df)
    print("---------------------------------------------------------\n")

## Associate a table for each proposal

In [ ]:
table_idx_of_proposal = {idx: None for idx in range(len(proposals))}
tables_to_be_assigned = [idx for idx in range(len(tables)) if len(titles[idx]) > 0]
for proposal_idx, proposal_text in enumerate(proposals):
    if proposal_idx in unanimous_idx:
        table_idx_of_proposal[proposal_idx] = -1
        continue
    table_idx = tables_to_be_assigned[0]
    if len(titles[table_idx]) > 0 and titles[table_idx].lower().replace("\uf0de", "").replace(" ", "") in proposal_text.lower().replace("\uf0de", "").replace(" ", ""):
        table_idx_of_proposal[proposal_idx] = table_idx
        tables_to_be_assigned.remove(table_idx)
        continue
tables_to_be_assigned += [idx for idx in range(len(tables)) if len(titles[idx]) == 0]
print(
    f"Tables left to be assigned: {len(tables_to_be_assigned)}\n"
    f"Proposals left to be assigned: {len([idx for idx, table_idx in table_idx_of_proposal.items() if table_idx is None])}"
)

In [ ]:
proposals_without_table = [
    proposals[idx]
    for idx in range(len(proposals))
    if table_idx_of_proposal[idx] is None
]
proposals_without_table

In [ ]:
proposals_without_owner_party = [
    proposal
    for proposal in proposals
    if not (
        any(f"({party})" in proposal.replace(" ", "") for party in deputies_per_party.keys()) 
        or "(GOV)" in proposal.replace(" ", "")
    )
]
proposals_without_owner_party

In [ ]:
for proposal in proposals_without_owner_party:
    idx = proposals.index(proposal)
    if idx == 0:
        continue
    if (table_idx_of_proposal[idx-1] is None) and (table_idx_of_proposal[idx] is not None):
        table_idx_of_proposal[idx-1] = table_idx_of_proposal[idx]
        proposals_without_table.remove(proposals[idx-1])
        proposals.remove(proposal)
        # shift all the indices of table_idx_of_proposal
        for i in range(idx, len(table_idx_of_proposal)-1):
            table_idx_of_proposal[i] = table_idx_of_proposal[i+1]
        table_idx_of_proposal.pop(list(table_idx_of_proposal.keys())[-1])
print(
    f"Tables left to be assigned: {len(tables_to_be_assigned)}\n"
    f"Proposals left to be assigned: {len(proposals_without_table)}"
)

In [ ]:
if len(proposals_without_table) == len(tables_to_be_assigned):
    pairs_to_assign = list(zip(proposals_without_table, tables_to_be_assigned))
    for proposal, table_idx in pairs_to_assign:
        proposal_idx = proposals.index(proposal)
        # only assign this table if it occurs between two nearest proposals' tables
        previous_table_idx = max(
            [
                table_idx_of_proposal[idx]
                for idx in range(proposal_idx)
                if table_idx_of_proposal[idx] is not None
            ]
        )
        next_table_idx = min(
            [
                table_idx_of_proposal[idx]
                for idx in range(proposal_idx, len(proposals))
                if table_idx_of_proposal[idx] is not None
                and table_idx_of_proposal[idx] != -1
            ]
        )
        if table_idx > previous_table_idx and table_idx < next_table_idx:
            table_idx_of_proposal[proposal_idx] = table_idx
            proposals_without_table.remove(proposal)
            tables_to_be_assigned.remove(table_idx)
print(
    f"Tables left to be assigned: {len(tables_to_be_assigned)}\n"
    f"Proposals left to be assigned: {len(proposals_without_table)}"
)

In [ ]:
table_idx_of_proposal